In [ ]:
import json
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import random
import torch
from torch.optim import Adam
import dgl
from datetime import datetime

from src import *
from pprint import pprint
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

data_dir = './data_passrefinder-p'

In [2]:
from argparse import ArgumentParser
parser = ArgumentParser()

import argparse
from pprint import pprint

parser = argparse.ArgumentParser(description="Parser for model configuration")

parser.add_argument('--feature_file', type=str, default=f"{data_dir}/data/feature_dict.json", help='feature json file')
parser.add_argument('--reuse_rate_file', type=str, default=f"{data_dir}/data/reuse_rate_dict.json", help='reuse rate json file')
# parser.add_argument('--setting', type=str, required=True, help='graph learning setting: inductive/transductive')
# parser.add_argument('--model_path', type=str, required=True, default="./model/", help='model file path')
parser.add_argument('--agg_type', type=str, default="attn", help='aggregation type')
# parser.add_argument('--nclient', type=int, default=10, help='number of clients')
# parser.add_argument('--valid', type=float, default=0.2, help='split ratio of validation set')
# parser.add_argument('--test', type=float, default=0.2, help='split ratio of test set')
parser.add_argument('--dropout', type=float, default=0.5, help='dropout ratio')
parser.add_argument('--random_seed', type=int, default=42, help='random seed for initialization')
parser.add_argument('--relu', type=float, default=0.2, help='ReLU threshold')
parser.add_argument('--reuse_th', type=float, default=0.5, help='threshold for reuse')
parser.add_argument('--train_batch_size', type=int, default=2048, help='batch size for training')
parser.add_argument('--eval_batch_size', type=int, default=2048, help='batch size for evaluation')
parser.add_argument('--embed_size', type=int, default=1024, help='size of the embedding layer')
parser.add_argument('--hidden_size', type=int, default=1024, help='size of the hidden layer')
parser.add_argument('--gnn_depth', type=int, default=2, help='depth of the GNN')
parser.add_argument('--max_lr', type=float, default=0.001, help='maximum learning rate')
parser.add_argument('--warmup', type=float, default=0.02, help='warmup ratio for learning rate')
parser.add_argument('--max_epoch', type=int, default=100, help='maximum number of epochs')
# parser.add_argument('--early_stop', type=int, default=40, help='early stopping epochs')
parser.add_argument('--device', type=int, default=0, help='GPU ID')

args = parser.parse_args(args=[])
pprint(args)

torch.manual_seed(args.random_seed)
torch.cuda.manual_seed(args.random_seed)
dgl.seed(args.random_seed)
np.random.seed(args.random_seed)
random.seed(args.random_seed)

Namespace(feature_file='./data_passrefinder-p/data/feature_dict.json', reuse_rate_file='./data_passrefinder-p/data/reuse_rate_dict.json', agg_type='attn', dropout=0.5, random_seed=42, relu=0.2, reuse_th=0.5, train_batch_size=2048, eval_batch_size=2048, embed_size=1024, hidden_size=1024, gnn_depth=2, max_lr=0.001, warmup=0.02, max_epoch=100, device=0)


In [3]:
with open(args.feature_file, 'r') as f:
    feature_dict = json.load(f)

with open(args.reuse_rate_file, 'r') as f:
    reuse_rate_dict = json.load(f)
    



g = construct_graph(feature_dict, reuse_rate_dict, args.reuse_th, hidden_relation=True)
g_dict, target_eids_dict, node_set_list = graph_split_base(g, args.random_seed)




P = PassREfinder_Base(g_dict, target_eids_dict, node_set_list, args)

Constructing graph...
Constructing graph... Done
Splitting graph...
Splitting graph... Done


In [ ]:
%%time
output_dir = "data_passrefinder-p/output/old_passrefinder/user/2025-03-27 13:09:23"
P.model = load_model(os.path.join(output_dir, 'model.pt'), P.model)
result, pair_to_score = P.evaluate(eval_ranking=True)
f1 = result['eval_f1']
precision = result['eval_precision']
recall = result['eval_recall']
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')

pprint(result['prec@k'])
pprint(result['nDCG@k'])

with open(os.path.join(output_dir, 'eval_results.json'), 'w') as f:
    json.dump(result, f)
with open(os.path.join(output_dir, 'pair_to_score.json'), 'w') as f:
    json.dump(pair_to_score, f)